In [160]:
import pandas as pd
import numpy as np

log = pd.read_csv("log.csv")
log = log.rename(columns=lambda x: x.strip())
log = log.drop(["start_time", "end_time"], axis=1)

In [161]:
log.head()

,subject_id,task_round,task_class,task_index,mouse_distance,mouse_click,keyboard_press,success,duration,prompt
0,9,0,0,0,6282,30,28,1,72.235,NaN
1,9,0,0,1,9917,18,48,1,62.757,NaN
2,9,0,0,2,4208,13,1,1,36.350,NaN
3,9,0,0,3,9320,8,0,1,33.527,NaN
4,9,0,0,4,12359,14,14,1,46.989,NaN


In [162]:
only_log = log.loc[(log["task_round"] == 1) & (log["success"] == 1)]
only_log = only_log.drop(["task_round","success"], axis=1)
only_log.head()

,subject_id,task_class,task_index,mouse_distance,mouse_click,keyboard_press,duration,prompt
13,9,0,0,1624,3,124,57.255,创建“旅行照片”文件夹，把3月8日创建的文件复制到里面
14,9,0,1,831,0,155,59.272,将“分享.jpg”的文件使用邮件发送给fakename@fakename.com，邮件名为“...
15,9,0,2,1116,0,60,28.643,换一张“山峰壁纸”
16,9,0,3,1929,2,103,40.265,分屏显示safari和提醒事项；开启专注模式
17,9,0,4,1750,1,59,21.993,截图全屏，投送出去


In [163]:
only_log

,subject_id,task_class,task_index,mouse_distance,mouse_click,keyboard_press,duration,prompt
13,9,0,0,1624,3,124,57.255,创建“旅行照片”文件夹，把3月8日创建的文件复制到里面
14,9,0,1,831,0,155,59.272,将“分享.jpg”的文件使用邮件发送给fakename@fakename.com，邮件名为“...
15,9,0,2,1116,0,60,28.643,换一张“山峰壁纸”
16,9,0,3,1929,2,103,40.265,分屏显示safari和提醒事项；开启专注模式
17,9,0,4,1750,1,59,21.993,截图全屏，投送出去
...,...,...,...,...,...,...,...,...
520,202,0,1,5333,6,18,152.216,将名字为’‘分享.jpg’‘的照片发邮件给fakename@fakemail.com，邮件主...
521,202,0,2,6747,2,1,27.440,找一张山峰的照片，换做桌面壁纸
522,202,0,3,738,0,4,49.442,分屏显示safari浏览器和提醒事项，然后开启专注模式
523,202,0,4,1161,3,3,48.944,截全屏，然后airdrop给最近的手机


In [164]:
import jieba
import re
def text_length(text):
  text = str(text)
  # text = re.sub(r'[，。”“"".；、]', '', text) 
  text_list = list(jieba.cut(text))
  print(text_list)
  length = len(text_list)
  # length = len(text)
  return length

only_log = only_log.assign(prompt_count = only_log["prompt"].apply(text_length))
only_log
prompt = only_log["prompt"]

['创建', '“', '旅行', '照片', '”', '文件夹', '，', '把', '3', '月', '8', '日', '创建', '的', '文件', '复制到', '里面']
['将', '“', '分享', '.', 'jpg', '”', '的', '文件', '使用', '邮件', '发送给', 'fakename', '@', 'fakename', '.', 'com', '，', '邮件', '名为', '“', '共享', '照片', '”']
['换', '一张', '“', '山峰', '壁纸', '”']
['分屏', '显示', 'safari', '和', '提醒', '事项', '；', '开启', '专注', '模式']
['截图', '全屏', '，', '投送', '出去']
['复制', '最近', '一个', '日程', '的', '位置', '到', '备忘录']
['制作', '一张', '带有', '标题', '账户', '密码', '登录', '组件', '的', '登录', '页面']
['填充', '海洋']
['制作', '4', '*', '4', '圆形', '矩阵', '，', '直径', '为', '16px', '间距', '为', '8px']
['替换', '全部', '字体', '为', 'roboto']
['为', '姓名', '替换', '真实', '名字']
['将', '图片', '的', '图层', '按', '尺寸', '命名']
['帮', '我', '把', '文件夹', '中', '所有', '3', '月', '8', '日', '创建', '的', '照片', '复制到', '一个', '名为', '“', '旅行', '照片', '”', '的', '新', '文件夹']
['把', '“', '分享', '”', '照片', '发到', 'fakename', '@', 'fakemail', '.', 'com', '的', '邮箱', '，', '标题', '为', '为', '共享', '照片']
['帮', '我', '设置', '一张', '山峰', '的', '桌面壁纸']
['给', '我', '分屏', '打开', 'safari', '和'

In [165]:
only_log = only_log.drop(["prompt"], axis=1).groupby("subject_id").sum()
only_log = only_log.assign(cost=only_log["mouse_distance"]/952+only_log["mouse_click"] * 0.1 + only_log["keyboard_press"] * 0.2)
only_log


,task_class,task_index,mouse_distance,mouse_click,keyboard_press,duration,prompt_count,cost
subject_id,,,,,,,,
2,6,30,16718,8,1092,341.584,155,236.760924
3,6,30,16922,20,785,397.007,134,176.775210
4,6,30,22396,15,1002,520.827,150,225.425210
5,6,30,20161,13,1097,493.094,161,241.877521
6,6,30,14137,8,1312,609.363,186,278.049790
7,6,30,43405,25,1333,573.731,147,314.693487
8,6,30,9394,5,840,319.289,122,178.367647
9,6,30,17619,13,968,376.677,112,213.407353
10,6,30,25313,8,1033,370.516,160,233.989286


In [166]:
scale = pd.read_csv("scale_origin.csv")
scale = scale[scale["task_round"] == 1]
scale

,subject_id,task_round,sus,effect,easy,easy_learn,satisfaction
0,202,1,77.5,52.6,64.6,26.6,45
2,206,1,57.5,49.6,58.8,24.0,48
4,205,1,60.0,48.8,64.0,26.6,49
6,204,1,85.0,49.6,60.2,23.8,48
8,203,1,67.5,51.8,58.6,24.8,46
10,201,1,50.0,54.6,67.2,26.6,49
12,13,1,87.5,47.6,67.2,24.6,48
14,15,1,80.0,54.6,64.8,26.6,48
16,16,1,72.5,45.8,58.4,26.6,49
18,14,1,75.0,54.6,67.2,26.6,49


In [167]:
mcat_data = pd.merge(only_log, scale, on="subject_id")
mcat_data = mcat_data.drop(["task_round", "task_class", "task_index"], axis=1)
mcat_data

,subject_id,mouse_distance,mouse_click,keyboard_press,duration,prompt_count,cost,sus,effect,easy,easy_learn,satisfaction
0,2,16718,8,1092,341.584,155,236.760924,80.0,47.6,60.8,26.6,49
1,3,16922,20,785,397.007,134,176.775210,70.0,44.0,58.8,26.6,43
2,4,22396,15,1002,520.827,150,225.425210,70.0,44.0,57.0,25.6,45
3,5,20161,13,1097,493.094,161,241.877521,60.0,52.6,56.6,24.6,48
4,6,14137,8,1312,609.363,186,278.049790,85.0,54.6,67.2,26.6,48
5,7,43405,25,1333,573.731,147,314.693487,82.5,53.6,67.2,26.6,49
6,8,9394,5,840,319.289,122,178.367647,62.5,46.0,54.8,23.0,42
7,9,17619,13,968,376.677,112,213.407353,65.0,43.8,51.0,21.0,40
8,10,25313,8,1033,370.516,160,233.989286,90.0,54.6,67.2,26.6,49
9,11,11575,10,753,259.809,90,163.758613,82.5,48.0,65.6,26.6,46


## 人口数据

In [168]:
popluation = pd.read_csv("population.csv")
popluation.columns

Index(['编号', '开始答题时间', '结束答题时间', '答题时长', '地理位置国家和地区', '地理位置省', '地理位置市',
       '自定义字段', 'IP', 'UA', 'Referrer', '1.《知情同意书》签字', '2.实验编号', '3.姓名',
       '4.您的出生年份是？', '5.您的性别是？', '6.到目前为止，您的最高学历(包括在读)是？', '7.您最后学习的专业是？:学科',
       '7.您最后学习的专业是？:专业名称', '8.您对 macOS 的熟悉情况如何？', '9.您对 Figma 的熟悉情况如何',
       '10.您的邮箱是?（选填）', '11.微信号/支付宝号（用于发放实验津贴）', 'Unnamed: 23'],
      dtype='object')

In [169]:
popluation = popluation.drop(['编号', '开始答题时间', '结束答题时间', '答题时长', '地理位置国家和地区', '地理位置省', '地理位置市',
       '自定义字段', 'IP', 'UA', 'Referrer', '1.《知情同意书》签字','10.您的邮箱是?（选填）', '11.微信号/支付宝号（用于发放实验津贴）', 'Unnamed: 23', '7.您最后学习的专业是？:学科',
       '7.您最后学习的专业是？:专业名称', '6.到目前为止，您的最高学历(包括在读)是？', "4.您的出生年份是？", ], axis=1)
popluation = popluation.rename({"2.实验编号": "subject_id", "3.姓名": "name", "8.您对 macOS 的熟悉情况如何？":"mac_os", "9.您对 Figma 的熟悉情况如何": "figma", "5.您的性别是？":"gender"}, axis=1)
popluation["mac_os"] = popluation["mac_os"].map({"D.我没用过/没听说过 macOS":0,"C.我是初级用户（只会基本操作，很多功能需要查阅资料）":1, "B.我是中级用户（熟悉各种快捷键和系统特性，能较轻松地完成日常活动）":2})
popluation["figma"] = popluation['figma'].map({"C.我是初级用户（只会基本操作，经常需要查阅 Figma 功能用法）":1, "B.我是中级用户（能够较流畅使用 Figma 产出设计图，但对高级功能了解有限）":2, "A.我是专家用户（能熟练使用组件系统、快捷键、插件系统等高级功能）":3})
popluation["gender"] = popluation["gender"].map({"A.男":1, "B.女":0})
popluation

,subject_id,name,gender,mac_os,figma
0,202,程艺明,0,1,1
1,206,余舟童,0,2,2
2,205,sim,0,2,1
3,204,王心宇,0,2,3
4,203,黄欧影,0,2,2
5,201,王文晔,0,1,1
6,13,陈超,1,2,2
7,15,徐瑶,0,2,1
8,16,张德寅,1,2,3
9,14,李萍,0,1,2


In [170]:
mcat_data = pd.merge(mcat_data, popluation, on="subject_id")

In [171]:
mcat_data.sort_values(by="mac_os", ascending=False)

,subject_id,mouse_distance,mouse_click,keyboard_press,duration,prompt_count,cost,sus,effect,easy,easy_learn,satisfaction,name,gender,mac_os,figma
10,12,17565,8,835,312.990,114,186.250630,85.0,50.8,61.2,24.6,46,赵婉莹,0,2,2
9,11,11575,10,753,259.809,90,163.758613,82.5,48.0,65.6,26.6,46,张珍妮,0,2,2
19,205,18335,13,917,396.456,119,203.959454,60.0,48.8,64.0,26.6,49,sim,0,2,1
18,204,39139,37,1158,507.725,175,276.412395,85.0,49.6,60.2,23.8,48,王心宇,0,2,3
17,203,53879,59,2372,794.629,293,536.895588,67.5,51.8,58.6,24.8,46,黄欧影,0,2,2
14,16,18697,26,1643,607.628,242,350.839706,72.5,45.8,58.4,26.6,49,张德寅,1,2,3
13,15,19539,11,848,455.093,115,191.224160,80.0,54.6,64.8,26.6,48,徐瑶,0,2,1
11,13,8151,5,1076,384.912,149,224.261975,87.5,47.6,67.2,24.6,48,陈超,1,2,2
1,3,16922,20,785,397.007,134,176.775210,70.0,44.0,58.8,26.6,43,孔蕾,0,2,1
20,206,6088,6,485,253.813,73,103.994958,57.5,49.6,58.8,24.0,48,余舟童,0,2,2


In [172]:
mcat_data[["prompt_count", "sus", "easy","easy_learn", "effect", "mac_os", "figma"]].corr(method="spearman")

,prompt_count,sus,easy,easy_learn,effect,mac_os,figma
prompt_count,1.000000,0.113644,0.014789,0.110671,0.154178,-0.176059,0.157763
sus,0.113644,1.000000,0.533620,0.195110,0.211289,-0.222638,0.187931
easy,0.014789,0.533620,1.000000,0.611904,0.682456,-0.553429,-0.208749
easy_learn,0.110671,0.195110,0.611904,1.000000,0.404260,-0.517598,-0.487735
effect,0.154178,0.211289,0.682456,0.404260,1.000000,-0.553721,-0.126799
mac_os,-0.176059,-0.222638,-0.553429,-0.517598,-0.553721,1.000000,0.350828
figma,0.157763,0.187931,-0.208749,-0.487735,-0.126799,0.350828,1.000000


In [173]:
from scipy import stats
buffer = mcat_data.drop(["subject_id", "name", "mouse_click", "mouse_distance", "keyboard_press", "figma"], axis=1)
corr, p = stats.spearmanr(buffer)
pd.DataFrame(corr, columns=buffer.columns, index=buffer.columns)

,duration,prompt_count,cost,sus,effect,easy,easy_learn,satisfaction,gender,mac_os
duration,1.000000,0.771679,0.784416,-0.098959,0.276841,0.090677,0.233523,0.278294,-0.104447,-0.029608
prompt_count,0.771679,1.000000,0.878857,0.113644,0.154178,0.014789,0.110671,0.373304,0.087067,-0.176059
cost,0.784416,0.878857,1.000000,0.069662,0.145292,0.011170,0.059822,0.417106,0.121854,-0.026318
sus,-0.098959,0.113644,0.069662,1.000000,0.211289,0.533620,0.195110,0.131442,-0.052359,-0.222638
effect,0.276841,0.154178,0.145292,0.211289,1.000000,0.682456,0.404260,0.500148,-0.710573,-0.553721
easy,0.090677,0.014789,0.011170,0.533620,0.682456,1.000000,0.611904,0.564228,-0.449179,-0.553429
easy_learn,0.233523,0.110671,0.059822,0.195110,0.404260,0.611904,1.000000,0.528841,-0.328471,-0.517598
satisfaction,0.278294,0.373304,0.417106,0.131442,0.500148,0.564228,0.528841,1.000000,-0.197748,-0.409381
gender,-0.104447,0.087067,0.121854,-0.052359,-0.710573,-0.449179,-0.328471,-0.197748,1.000000,0.176383
mac_os,-0.029608,-0.176059,-0.026318,-0.222638,-0.553721,-0.553429,-0.517598,-0.409381,0.176383,1.000000


In [174]:
def highlight(val):
    color = 'red' if val < 0.05 and val > 0 else 'white'
    return 'color: %s' % color
  
p = pd.DataFrame(p, columns=buffer.columns, index=buffer.columns)
p.apply(lambda x: x.map(lambda y: round(y, 3))).style.applymap(highlight)


,duration,prompt_count,cost,sus,effect,easy,easy_learn,satisfaction,gender,mac_os
duration,0.000000,0.000000,0.000000,0.670000,0.224000,0.696000,0.308000,0.222000,0.652000,0.899000
prompt_count,0.000000,0.000000,0.000000,0.624000,0.505000,0.949000,0.633000,0.096000,0.707000,0.445000
cost,0.000000,0.000000,0.000000,0.764000,0.530000,0.962000,0.797000,0.060000,0.599000,0.910000
sus,0.670000,0.624000,0.764000,0.000000,0.358000,0.013000,0.397000,0.570000,0.822000,0.332000
effect,0.224000,0.505000,0.530000,0.358000,0.000000,0.001000,0.069000,0.021000,0.000000,0.009000
easy,0.696000,0.949000,0.962000,0.013000,0.001000,0.000000,0.003000,0.008000,0.041000,0.009000
easy_learn,0.308000,0.633000,0.797000,0.397000,0.069000,0.003000,0.000000,0.014000,0.146000,0.016000
satisfaction,0.222000,0.096000,0.060000,0.570000,0.021000,0.008000,0.014000,0.000000,0.390000,0.065000
gender,0.652000,0.707000,0.599000,0.822000,0.000000,0.041000,0.146000,0.390000,0.000000,0.444000
mac_os,0.899000,0.445000,0.910000,0.332000,0.009000,0.009000,0.016000,0.065000,0.444000,0.000000


In [175]:
stats.shapiro(mcat_data["prompt_count"])

ShapiroResult(statistic=0.8926445245742798, pvalue=0.025262491777539253)

In [176]:
stats.describe(mcat_data["prompt_count"])


DescribeResult(nobs=21, minmax=(73, 293), mean=146.38095238095238, variance=2542.0476190476193, skewness=1.2751384330844044, kurtosis=1.915418899054222)

In [177]:
stats.spearmanr(mcat_data["sus"], mcat_data["mac_os"] + mcat_data["figma"])

SignificanceResult(statistic=-0.031648003354182164, pvalue=0.8916771768422115)

In [178]:
def summary_column(column):
    data = column
    n = len(data)
    mean = np.mean(data)
    std = np.std(data)
    std_error = std / np.sqrt(n)
    ci = stats.t.interval(0.95, n-1, loc=mean, scale=std_error)
    ci_lower = ci[0]
    ci_upper = ci[1]

    minimum = np.min(data)
    maximum = np.max(data)
    return pd.DataFrame({'column_name': column.name, 'sample_size': n, 'mean': mean, 
                                 'std': std, 'std_error': std_error, 'ci_lower': ci_lower, 
                                 'ci_upper': ci_upper, 'min': minimum, 'max': maximum}, index=[0])

def summary(df):
    summary_df = pd.DataFrame(columns=['column_name', 'sample_size', 'mean', 'std', 'std_error', 'ci_lower', 'ci_upper', 'min', 'max'])
    dfs = []
    
    for column in df.columns:
        dfs.append(summary_column(column))
    
    summary_df = pd.concat(dfs, ignore_index=True)
    
    return summary_df



In [179]:
res = summary_column(mcat_data["prompt_count"]).round(4)

In [180]:
res.to_csv("prompt_describe.csv", index=False)

In [181]:
def pval_df(df, spearman=False):
    pval_matrix = []
    for col1 in df.columns:
        pval_row = []
        for col2 in df.columns:
            # 使用scipy.stats模块中的pearsonr()函数，返回两个值：相关系数和
            if spearman == False:
                pval = stats.pearsonr(df[col1], df[col2])[1]
            else:
                pval = stats.spearmanr(df[col1], df[col2])[1]
            pval_row.append(pval)
        pval_matrix.append(pval_row)
    return pd.DataFrame(pval_matrix, index=df.columns, columns=df.columns)

In [213]:
# 文本分词和交互数据的 Spearman 相关系数
data = mcat_data.loc[:, ["prompt_count", "duration", "cost"]]
data = mcat_data.loc[:, ["prompt_count", "duration", "cost", "sus", "easy", "easy_learn", "satisfaction"]]
# corr = data.corr(method="pearson")
corr = data.corr(method="spearman")
# corr = pd.DataFrame(corr,columns=data.columns).reset_index()
corr 
# print corr as csv

,prompt_count,duration,cost,sus,easy,easy_learn,satisfaction
prompt_count,1.000000,0.771679,0.878857,0.113644,0.014789,0.110671,0.373304
duration,0.771679,1.000000,0.784416,-0.098959,0.090677,0.233523,0.278294
cost,0.878857,0.784416,1.000000,0.069662,0.011170,0.059822,0.417106
sus,0.113644,-0.098959,0.069662,1.000000,0.533620,0.195110,0.131442
easy,0.014789,0.090677,0.011170,0.533620,1.000000,0.611904,0.564228
easy_learn,0.110671,0.233523,0.059822,0.195110,0.611904,1.000000,0.528841
satisfaction,0.373304,0.278294,0.417106,0.131442,0.564228,0.528841,1.000000


In [226]:
print(rename(pval_df(corr).round(4)).to_csv())


,文本分词长度,任务时长,操作成本,SUS 评分,USE 易用性,USE 易学性,USE 满意度
文本分词长度,0.0,0.0044,0.0001,0.0976,0.0016,0.0731,0.6923
任务时长,0.0044,0.0,0.0041,0.0174,0.0164,0.2603,0.7401
操作成本,0.0001,0.0041,0.0,0.0822,0.003,0.0709,0.7905
SUS 评分,0.0976,0.0174,0.0822,0.0,0.19,0.9578,0.4678
USE 易用性,0.0016,0.0164,0.003,0.19,0.0,0.0728,0.4182
USE 易学性,0.0731,0.2603,0.0709,0.9578,0.0728,0.0,0.2729
USE 满意度,0.6923,0.7401,0.7905,0.4678,0.4182,0.2729,0.0



In [216]:
p = pval_df(data)
p

,prompt_count,duration,cost,sus,easy,easy_learn,satisfaction
prompt_count,0.000000e+00,9.371166e-07,2.455124e-09,0.886284,0.745673,0.674780,2.983648e-01
duration,9.371166e-07,2.557517e-147,1.905594e-06,0.577940,0.818340,0.444240,2.920133e-01
cost,2.455124e-09,1.905594e-06,0.000000e+00,0.931485,0.736523,0.982932,3.556820e-01
sus,8.862843e-01,5.779400e-01,9.314847e-01,0.000000,0.039200,0.274411,3.810489e-01
easy,7.456729e-01,8.183402e-01,7.365233e-01,0.039200,0.000000,0.000208,8.164557e-04
easy_learn,6.747805e-01,4.442397e-01,9.829324e-01,0.274411,0.000208,0.000000,2.809092e-03
satisfaction,2.983648e-01,2.920133e-01,3.556820e-01,0.381049,0.000816,0.002809,1.851840e-144


In [212]:
import pyperclip
def rename(df:pd.DataFrame):
    df = df.rename({"prompt_count": "文本分词长度", "duration": "任务时长", "cost": "操作成本", "sus":"SUS 评分","effect": "USE 有效性", "easy": "USE 易用性", "easy_learn": "USE 易学性", "satisfaction": "USE 满意度"}, axis=1)
    df = df.rename({"prompt_count": "文本分词长度", "duration": "任务时长", "cost": "操作成本", "sus":"SUS 评分","effect": "USE 有效性", "easy": "USE 易用性", "easy_learn": "USE 易学性", "satisfaction": "USE 满意度"}, axis=0)
    # df[""] = df["index"].map({"prompt_count": "文本分词长度", "duration": "任务时长", "cost": "操作成本", "sus":"SUS 评分","effect": "USE 有效性", "easy": "USE 易用性", "easy_learn": "USE 易学性", "satisfaction": "USE 满意度"})
    # df 第一行替换

    return df



In [187]:
corr = rename(corr)
print(corr)
# pyperclip.copy(corr.to_markdown(index=False))

                文本分词长度      任务时长      操作成本    SUS 评分   USE 易用性   USE 易学性  \
prompt_count  1.000000  0.771679  0.878857  0.113644  0.014789  0.110671   
duration      0.771679  1.000000  0.784416 -0.098959  0.090677  0.233523   
cost          0.878857  0.784416  1.000000  0.069662  0.011170  0.059822   
sus           0.113644 -0.098959  0.069662  1.000000  0.533620  0.195110   
easy          0.014789  0.090677  0.011170  0.533620  1.000000  0.611904   
easy_learn    0.110671  0.233523  0.059822  0.195110  0.611904  1.000000   
satisfaction  0.373304  0.278294  0.417106  0.131442  0.564228  0.528841   

               USE 满意度  
prompt_count  0.373304  
duration      0.278294  
cost          0.417106  
sus           0.131442  
easy          0.564228  
easy_learn    0.528841  
satisfaction  1.000000  


In [188]:
p = rename(p)
print(p)
pyperclip.copy(p.to_markdown(index=False))

                    文本分词长度           任务时长          操作成本    SUS 评分   USE 易用性  \
prompt_count  0.000000e+00   9.371166e-07  2.455124e-09  0.886284  0.745673   
duration      9.371166e-07  2.557517e-147  1.905594e-06  0.577940  0.818340   
cost          2.455124e-09   1.905594e-06  0.000000e+00  0.931485  0.736523   
sus           8.862843e-01   5.779400e-01  9.314847e-01  0.000000  0.039200   
easy          7.456729e-01   8.183402e-01  7.365233e-01  0.039200  0.000000   
easy_learn    6.747805e-01   4.442397e-01  9.829324e-01  0.274411  0.000208   
satisfaction  2.983648e-01   2.920133e-01  3.556820e-01  0.381049  0.000816   

               USE 易学性        USE 满意度  
prompt_count  0.674780   2.983648e-01  
duration      0.444240   2.920133e-01  
cost          0.982932   3.556820e-01  
sus           0.274411   3.810489e-01  
easy          0.000208   8.164557e-04  
easy_learn    0.000000   2.809092e-03  
satisfaction  0.002809  1.851840e-144  


In [189]:
stats.pearsonr(mcat_data["prompt_count"], mcat_data["sus"])

PearsonRResult(statistic=0.03323326559427906, pvalue=0.8862843251166858)

In [190]:
data = mcat_data.loc[:, ["prompt_count", "effect", "easy", "easy_learn", "satisfaction"]]
corr, p = stats.spearmanr(data)
corr = pd.DataFrame(corr,columns=data.columns, index=data.columns)
corr

,prompt_count,effect,easy,easy_learn,satisfaction
prompt_count,1.000000,0.154178,0.014789,0.110671,0.373304
effect,0.154178,1.000000,0.682456,0.404260,0.500148
easy,0.014789,0.682456,1.000000,0.611904,0.564228
easy_learn,0.110671,0.404260,0.611904,1.000000,0.528841
satisfaction,0.373304,0.500148,0.564228,0.528841,1.000000


In [191]:
p = pd.DataFrame(p, columns=data.columns, index=data.columns)
p

,prompt_count,effect,easy,easy_learn,satisfaction
prompt_count,0.000000,5.045958e-01,9.492688e-01,6.329526e-01,0.095549
effect,0.504596,3.532105e-150,6.532211e-04,6.912516e-02,0.020948
easy,0.949269,6.532211e-04,3.532105e-150,3.198944e-03,0.007714
easy_learn,0.632953,6.912516e-02,3.198944e-03,3.532105e-150,0.013706
satisfaction,0.095549,2.094750e-02,7.714009e-03,1.370606e-02,0.000000


In [192]:
res =  pd.concat([corr.iloc[[0]], p.iloc[[0]]]).drop(["prompt_count"],axis=1)
res = res.rename({"effect": "USE 有效性", "easy": "USE 易用性", "easy_learn": "USE 易学性", "satisfaction": "USE 满意度"}, axis = 1)
print(res)
pyperclip.copy(res.to_markdown())

               USE 有效性   USE 易用性   USE 易学性   USE 满意度
prompt_count  0.154178  0.014789  0.110671  0.373304
prompt_count  0.504596  0.949269  0.632953  0.095549


In [193]:
data = mcat_data.loc[:, ["mac_os", "sus","effect", "easy", "easy_learn", "satisfaction"]]
corr, p = stats.spearmanr(data)
corr = pd.DataFrame(corr,columns=data.columns, index=data.columns)
p = pd.DataFrame(p, columns=data.columns, index=data.columns)

In [194]:
res =  pd.concat([corr.iloc[[0]], p.iloc[[0]]]).drop(["mac_os"],axis=1)
res = res.rename({"sus":"SUS 评分","effect": "USE 有效性", "easy": "USE 易用性", "easy_learn": "USE 易学性", "satisfaction": "USE 满意度"}, axis = 1)
res = res.assign(统计量=["rho", "p"])
cols = list(res.columns)
cols = [cols[-1]] + cols[:-1]
res = res[cols]

In [229]:
res

,sus,effect,easy,easy_learn,satisfaction
mac_os,-0.222638,-0.553721,-0.553429,-0.517598,-0.409381
mac_os,0.332032,0.009208,0.009253,0.016253,0.065348


In [230]:
print(rename(res).round(4).to_csv())

,SUS 评分,USE 有效性,USE 易用性,USE 易学性,USE 满意度
mac_os,-0.2226,-0.5537,-0.5534,-0.5176,-0.4094
mac_os,0.332,0.0092,0.0093,0.0163,0.0653



In [197]:
res =  pd.concat([corr.iloc[[0]], p.iloc[[0]]]).drop(["mac_os"],axis=1)
res

,sus,effect,easy,easy_learn,satisfaction
mac_os,-0.222638,-0.553721,-0.553429,-0.517598,-0.409381
mac_os,0.332032,0.009208,0.009253,0.016253,0.065348


In [232]:
corr, p = stats.spearmanr(mcat_data["prompt_count"], mcat_data["mac_os"])

In [233]:
corr.round(4)

-0.1761

In [234]:
p.round(4)

0.4452